In [1]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocess as mp
import glob
import time
from tqdm import tqdm
import os
import sys
import pandas as pd
from eumap.misc import find_files, nan_percentile, GoogleSheet, ttprint
from eumap.raster import read_rasters, save_rasters
from eumap.mapper import SpaceOverlay
import geopandas as gpd
from pathlib import Path
from minio import Minio
import rasterio
import pyproj
from shapely.geometry import Point
import requests
import warnings
warnings.filterwarnings('default')

# os.environ['PROJ_LIB'] = '/opt/conda/share/proj'
folder = '/home/opengeohub/xuemeng/work_xuemeng/soc'
# /home/opengeohub/.local/bin

### check what data needs to be overlayed

In [2]:
from shapely.geometry import Point
from geopandas import gpd  

added_covar = 1 # wether overlay from scratch or from overlayed version

if added_covar:
    df = pd.read_csv(f'{folder}/data/002.2_data_whole.csv', low_memory=False)
else:
    df = pd.read_csv(f'{folder}/data/000.2_soil.full_qa.controlled.csv', low_memory=False)

geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

df4326 = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
df3035 = df4326.to_crs("EPSG:3035")


### generate overlay links
- read in the files specified by Google Sheet
- convert the files into readable linkes for overlay

In [15]:
# read in potential usable overlay files
key_file = '/home/opengeohub/xuemeng/work_xuemeng/gaia-319808-913d36b5fca4.json'
url = 'https://docs.google.com/spreadsheets/d/1eIoPAvWM5jrhLrr25jwguAIR0YxOh3f5-CdXwpcOIz8/edit#gid=0'

gsheet = GoogleSheet(key_file, url)
covar = gsheet.covar


/opt/conda/lib/python3.8/site-packages/eumap/misc.py:361: ResourceWarning: unclosed <ssl.SSLSocket fd=65, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.2', 53018), raddr=('172.217.23.202', 443)>
  self._read_gsheet()


In [19]:
a = covar['path'].iloc[0]
a

'http://192.168.1.30:8333/ai4sh/blue/blue_glad.landsat.seasconv_m_30m_s_{year}{start_m}_{year}{end_m}_eu_epsg.3035_v20231211.tif'

'http://192.168.1.30:8333/ai4sh/vrt/blue_glad.landsat.seasconv_m_30m_s_{year}{start_m}_{year}{end_m}_eu_epsg.3035_v20231211.tif.vrt'

In [5]:
# function to generate file paths by year, and check if the urls are valid
def generate_overlay_path(row,year,filt=None):
    # check if the covariates resolution is 30m, otherwise use .vrt
    if row['spatial resolution'] != '30m':
        row_path = 'http://192.168.1.30:8333/ai4sh/vrt/' + row['path'].split('/')[-1] +'.vrt'
            
    # determine if static variable
    if row['temporal resolution'] == 'static':
        return [row['path']],[row['path']]
    
    if row['temporal resolution'] == 'long term':
        perc_list = row['perc'].split(',')
        output_paths = [row['path'].replace('{perc}', perc) for perc in perc_list]
        return output_paths, output_paths
        
    # determine if the year is ahead of the availibility of the variable
    if year>int(row['end year']):
        year = int(row['end year'])
    if year<int(row['start year']):
        year = int(row['start year'])
    # determine if it's an annual variable or (bi)monthly variable
    if '{start_m}' not in row['path']:
        output_paths = [row['path'].replace('{year}',f'{int(year)}')]
    else:
        output_paths = []
        start_list = row['start_m'].split(', ')
        end_list = row['end_m'].split(', ')
        output_paths = [row['path'].replace('{year}',f'{int(year)}').replace('{start_m}',start_list[i]).replace('{end_m}',end_list[i]) for i in range(len(end_list))]
    
    if '{perc}' in row['path']:
        perc_list = row['perc'].split(',')
        output_paths = [p.replace('{perc}', perc) for p in output_paths for perc in perc_list]
        
    if (row['leap year'] == '1') & (year%4==0):
        output_paths = [p.replace('0228', '0229') if '0228' in p else p for p in output_paths]
    
    return output_paths, [i.replace(str(int(year)),'{year}') for i in output_paths]
    
def check_path(url):
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        # Check if the status code is not 200 (OK). You might want to specifically check for 404 or other error codes.
        if response.status_code == 404:
            print(f"{url} returned HTTP 404 Not Found")
            return url
        elif response.status_code != 200:
            print(f"{url} returned HTTP {response.status_code}")
            return url
        return None  # URL is fine (HTTP 200), or you might want to handle redirections (HTTP 3xx) separately if needed.
    except requests.RequestException as e:
        print(f"Failed to retrieve {url}: {str(e)}")
        return url
    
# # check function validity
# # generate paths
# paths = []
# for index,row in covar.iterrows():
#     paths.extend(generate_overlay_path(row,2000))
    
pathl = []
namel = []
year = 2000
for index,row in covar.iterrows():
    if added_covar: 
        if row['need update in overlay']=='1':
            paths, names = generate_overlay_path(row, year)
            pathl.extend(paths)
            namel.extend(names)
    else:
        paths, names = generate_overlay_path(row, year)
        pathl.extend(paths)
        namel.extend(names)
for i in pathl:
    check_path(i)

### static layers - overlay

In [7]:
from skmap.mapper import SpaceOverlay, SpaceTimeOverlay
temp = SpaceOverlay(
    points=df3035, 
    fn_layers=sta_path, verbose=True
).run()
temp

# temp = temp.drop(columns=['overlay_id', 'geometry'])


ttprint(f'finish overlaying for static, size: {len(temp)}, column num: {len(sta_path)}')
# temp.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc/overlay_intermediate/temp_overlay_static.csv', index=False)

42


[]

### dynamic layers - overlay

In [ ]:
# # creat sub-samples to test
# samp_points_f_sub = []
# for _,rows in df3035.groupby('time'):
#     samp_points_f_sub.append(rows.sample(int(rows.shape[0] * 0.005)))
# samp_points_f_sub = pd.concat(samp_points_f_sub)
# print(len(samp_points_f_sub))

# samp_points_f_sub = samp_points_f_sub.loc[samp_points_f_sub['time'].isin([2009,2010,2016])]
# print(len(samp_points_f_sub))

In [9]:
drop = []

for ii in df.columns:
    if 'lithology' in ii:
        drop.append(ii)
        
dff = df.drop(columns=drop)


In [14]:
for ii in dff.columns:
    print(ii)

id
lat
lon
time
ref
nuts0
oc
ph_h2o
ph_cacl2
bulk_density
clay
silt
sand
caco3
N
K
P
CEC
EC
oc_qa
N_qa
caco3_qa
bulk_density_qa
clay_qa
silt_qa
sand_qa
ph_h2o_qa
ph_cacl2_qa
P_qa
K_qa
EC_qa
CEC_qa
thermal_glad.landsat.seasconv_m_30m_s_{year}0901_{year}1031_eu_epsg.3035_v20231211
thermal_glad.landsat.seasconv_m_30m_s_{year}1101_{year}1231_eu_epsg.3035_v20231211
thermal_glad.landsat.seasconv_m_30m_s_{year}0101_{year}0228_eu_epsg.3035_v20231211
thermal_glad.landsat.seasconv_m_30m_s_{year}0701_{year}0831_eu_epsg.3035_v20231211
thermal_glad.landsat.seasconv_m_30m_s_{year}0301_{year}0430_eu_epsg.3035_v20231211
thermal_glad.landsat.seasconv_m_30m_s_{year}0501_{year}0630_eu_epsg.3035_v20231211
blue_glad.landsat.seasconv_m_30m_s_{year}0701_{year}0831_eu_epsg.3035_v20231211
blue_glad.landsat.ard2.seasconv.m.yearly_p25_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20231127
thermal_glad.landsat.ard2.seasconv.m.yearly_p50_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20231127
thermal_glad.landsat.ard2.seas

In [ ]:
from skmap.mapper import SpaceOverlay, SpaceTimeOverlay

dff['ref_date'] = pd.to_datetime(dff.time, format='%Y')
dff_overlaid = SpaceTimeOverlay(
    points=dff, col_date='ref_date', fn_layers=mendl, verbose=True
).run()

ttprint(f'finish overlaying for static, size: {len(dff)}, column num: {len(mendl)}') 

### assemble the overlayed annual datasets
- read in the overlayed soil data (with covariates) for each year
- combine them into a whole dataset

In [16]:
## read in 3035 datasets
mended = ''
tl = []
for year in np.arange(2000,2023,1):
    temp = pd.read_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}{mended}_3035.csv',low_memory=False)
    print(f'{year}, {len(temp.columns)}, {len(temp)}')
    tl.append(temp)

df3035 = pd.concat(tl)
print(f'whole 3035, cols:{len(df3035.columns)}, size:{len(df3035)}')


2000, 449, 9869
2001, 449, 2245
2002, 449, 3124
2003, 449, 3972
2004, 449, 1589
2005, 449, 3332
2006, 449, 3764
2007, 449, 4735
2008, 449, 1929
2009, 449, 1602
2010, 449, 2724
2011, 449, 3385
2012, 449, 3808
2013, 449, 3278
2014, 449, 2404
2015, 449, 1230
2016, 449, 2109
2017, 449, 1950
2018, 449, 1535
2019, 449, 2033
2020, 449, 1009
2021, 449, 1855
2022, 449, 549
whole 3035, cols:449, size:64030


In [3]:
col = 'cropland.extent_glad.interpolate_p_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20240604'
for year in np.arange(2000,2023,1):
    temp = df3035.loc[df3035['time']==year]
    aaa = temp[col].isna().sum()/len(temp)
    print(aaa, year, temp[col].min())
    
df3035.loc[df3035[col].isna(),col] = 0

0.7101201338199513 2000 100.0
0.8049620411677233 2001 100.0
0.6483188257381806 2002 100.0
0.4095724975920575 2003 100.0
0.7522551967577461 2004 25.0
0.7871717871717872 2005 50.0
0.6450478011472275 2006 25.0
0.6639189698830882 2007 100.0
0.6091129099557707 2008 25.0
0.5784442217253875 2009 50.0
0.5128953771289537 2010 25.0
0.5467022245349291 2011 100.0
0.6755429360755122 2012 25.0
0.4654657457532201 2013 50.0
0.6197848954024348 2014 25.0
0.578684429641965 2015 100.0
0.6435062125239538 2016 25.0
0.4782553809253298 2017 50.0
0.5796868446805536 2018 25.0
0.7391304347826086 2019 100.0
0.7472549019607844 2020 100.0
0.6805989344111704 2021 100.0
0.8269581056466302 2022 100.0


In [5]:
dff = df3035
meta_list = ['id', 'lat', 'lon', 'time', 'hzn_top', 'hzn_btm', 'ref', 'nuts0', 'oc',
       'ph_h2o', 'ph_cacl2', 'bulk_density', 'clay', 'silt', 'sand', 'caco3',
       'N', 'K', 'P', 'CEC', 'EC', 'oc_qa', 'N_qa', 'caco3_qa',
       'bulk_density_qa', 'clay_qa', 'silt_qa', 'sand_qa', 'ph_h2o_qa',
       'ph_cacl2_qa', 'P_qa', 'K_qa', 'EC_qa', 'CEC_qa', 'geometry']
### check covariates availability
drop_list = []
for col in dff.columns:
    if col in meta_list:
        continue
    if (dff[col].isna().sum()/len(dff))>0.02:
        
        if 'longterm' not in col:
            print(col, dff[col].isna().sum())
            drop_list.append(col)
        
print(f'remove covariates with more than 2% data unavailable')
dff = dff.drop(columns = drop_list)

fgd_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531 264869
fcf_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531 198538
lgd_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531 264869
surface.water_jrc.gswe_p_30m_s_2000_2018_go_epsg.4326_v01232022 394643
remove covariates with more than 2% data unavailable


In [6]:
dff = dff.drop(columns=['geometry'])
# dff.to_csv(f'{folder}/data/001_covar_overlayed.csv',index=False)

### Assign spatial blocking ID

In [9]:
# create a tiling system first
from eumap.parallel import TilingProcessing
from pathlib import Path
import rasterio
from shapely.geometry import box
import numpy as np
import pandas as pd
import geopandas as gpd

# # create tiling system
# raster_layer_fn = f'http://192.168.1.30:8333/ai4sh-landmasked/ndvi/ndvi_glad.landast.ard2.seasconv.m.yearly_p75_30m_s_20220101_20221231_eu_epsg.3035_v20231127.tif'
# ds = rasterio.open(raster_layer_fn)
# tiles_size = ds.transform[0] * 1000 # 30m -> 30km
# tiling_system = TilingProcessing.generate_tiles(tiles_size, extent=ds.bounds, crs=ds.crs, raster_layer_fn=raster_layer_fn)
# tiling_system = tiling_system.to_crs("EPSG:4326")
# tiling_system = tiling_system[['tile_id','geometry']]
# tiling_system.to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg',  driver="GPKG")
# # tiling_system[tiling_system['raster_mode_count'] > 0].to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg.gpkg',  driver="GPKG")

dff = temp
from shapely.geometry import Point
tiles = gpd.read_file(f'{folder}/data/000_tiles.rolf_epsg.3035.gpkg')

# dff = pd.read_csv(f'{folder}/data/001_covar_overlayed.csv',low_memory=False)
gdf = gpd.GeoDataFrame(dff, geometry=gpd.points_from_xy(dff.lon, dff.lat))
gdf.crs = 'EPSG:4326'
gdf = gdf.to_crs(tiles.crs)

# join spatial
joined_gdf = gpd.sjoin(gdf, tiles, how="left", op='within')
joined_gdf = joined_gdf.drop(columns=['geometry', 'index_right', 'left', 'top', 'right', 'bottom','row_index', 'col_index'])
joined_gdf = joined_gdf.rename(columns={'id_right':'tile_id','id_left':'id'})

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [12]:
joined_gdf.to_csv('/mnt/primus/xuemeng_tmp_harbour/soc/data/003_bedrock_whole.csv',index=False)
# joined_gdf.to_csv(f'{folder}/data/001_covar_overlayed.csv',index=False)

### epsg 4326

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

# # epsg 4326 overlay
# co4326 = covar.loc[covar['epsg']=='4326']
# path_ori = [i.split('/')[-1][0:-4] for i in co4326['path']]

# for year in np.arange(2000,2024,1):
#     pathl = []
#     namel = []
#     for index,row in co4326.iterrows():
#         paths, names = generate_overlay_path(row, year)
#         pathl.extend(paths)
#         namel.extend(names)
#         # path4326.extend(generate_overlay_path(row,year))
        
# #     for i in pathl:
# #         check_path(i)
        
#     path_stem = [i.split('/')[-1][0:-4] for i in pathl]
#     namel = [i.split('/')[-1][0:-4] for i in namel]
#     name_mapping = dict(zip(path_stem,namel))
    
#     df_overlay = df4326.loc[df4326['time']==year]
#     if len(df_overlay)==0:
#         print(f'no data for year {year}')
#         continue
        
#     ttprint(f'start overlaying for year {str(int(year))}, size: {len(df_overlay)}')
#     pathl = [Path(ii) for ii in pathl]
#     dfo = SpaceOverlay(df_overlay, fn_layers=pathl, max_workers=90, verbose=False)
#     temp = dfo.run()
    
#     temp = temp.rename(columns=name_mapping)
#     temp=temp.drop(columns=['overlay_id'])
#     # temp = pd.read_csv(f'/mnt/inca/soc_eu_model/overlay_intermediate/dft_{str(int(tt))}_4326.csv',index=False)
#     temp.to_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_4326.csv',index=False)
#     ttprint(f'finish overlaying for year {str(int(year))}')


# # read in 4326 datasets
# tl = []

# for year in np.arange(2000,2023,1):
#     temp = pd.read_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_4326.csv',low_memory=False)
#     temp = temp.rename(columns=name_mapping)
#     print(f'{year}, {len(temp.columns)}, {len(temp)}')
#     tl.append(temp)

# df4326 = pd.concat(tl)
# print(f'whole 4326, {len(df4326.columns)}, {len(df4326)}')


# # merge to merge
# cols_list = df3035.columns.values.tolist()
# meta_list = ['lat', 'lon', 'oc', 'ph_h2o', 'ph_cacl2', 'bulk_density', 'clay','silt', 'sand', 'caco3','N',
#              'K', 'P', 'CEC', 'EC', 'nuts0', 'time','hzn_top','hzn_btm','ref','sample_id']
# dff = pd.merge(df3035,df4326,on = meta_list, how='inner')
